## <center><strong>Notebook<span style= "color: #51FCC6"> Describtion

## <center><strong>Importing<span style= "color: #48E0DC"> Packeges

In [1]:
import time
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
from IPython.display import set_matplotlib_formats
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
COLORS = ["#51fcc6", "#48e0dc", "#5cd3f7", "#4895e0", "#517afc"]

%matplotlib inline

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings("ignore")

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_7728\2138414856.py:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center><strong>Setting up the<span style= "color: #5CD3F7"> Web scrapers

In [49]:
data_jobs_titles = ["data-entry", "data-processing", "data-engineering"]
                    # "data-science", "data-analytics", "data-visualization"]

upwork_types = ["profiles", "jobs", "services"]
pph_types = ["hire-freelancers", "freelance-jobs", "services"]
guru_service_types = ["freelancers", "jobs"]


def scrape_page(url: str, retrieve_new_url= False) -> BeautifulSoup:
    driver = webdriver.Chrome()
    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    
    # Wait until the page is fully loaded
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    html = driver.page_source

    if retrieve_new_url:
        
        new_url = driver
        driver.quit()
        
        return BeautifulSoup(html), new_url
        
    driver.quit()

    return BeautifulSoup(html)

In [50]:
# The `type` param indicates what is the nature of the job is it hiring freelancers 
# or paying services or othr thing based on the platform.


def upwork_scraper(page, job_title, type):

    global upwork_url

    if  type == 'services':

        upwork_url =  f"https://www.upwork.com/services/search?nbs=1&q={job_title}"

        driver = webdriver.Chrome()
        driver.get(upwork_url)

        if page > 1:
            for i in range(page):

                try:
                    wait = WebDriverWait(driver, 10)
                    element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.up-btn.up-btn-default.mx-0.load-more-btn.nuxt-link-active')))
                    element.click()
                    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

                except:
                    raise  KeyError

        html = driver.page_source
        driver.quit()
        
        return  html

    elif type == "profiles" or type == "jobs":
        
        upwork_url = f"https://www.upwork.com/search/{type}/?page={page}&q={job_title}&user_pref=1"
        html = scrape_page(upwork_url)

        # The code below checks if there's any element that appeared in the page that indicates
        # that the scraper exceed the maxx page.

        if html.find("h2", class_="mt-30 mb-20 text-muted"):

            # We are going to use the keyerror to indicate that the page param is more than the
            # max possible page whitch we will use to control the other scrapers.
            
            raise  KeyError

        elif html.find("h2", class_="mt-30 text-muted"):
            raise  KeyError

        return html

In [51]:
freelancer_scraper(9999, "data-science")

<html class="is-mobile-first" lang="en"><head><style>.pac-container{background-color:#fff;position:absolute!important;z-index:1000;-webkit-border-radius:2px;-moz-border-radius:2px;border-radius:2px;border-top:1px solid #d9d9d9;font-family:Arial,sans-serif;-webkit-box-shadow:0 2px 6px rgba(0,0,0,.3);-moz-box-shadow:0 2px 6px rgba(0,0,0,.3);box-shadow:0 2px 6px rgba(0,0,0,.3);-moz-box-sizing:border-box;-webkit-box-sizing:border-box;box-sizing:border-box;overflow:hidden}.pac-logo:after{content:"";padding:1px 1px 1px 0;height:18px;-webkit-box-sizing:border-box;-moz-box-sizing:border-box;box-sizing:border-box;text-align:right;display:block;background-image:url(https://maps.gstatic.com/mapfiles/api-3/images/powered-by-google-on-white3.png);background-position:right;background-repeat:no-repeat;-webkit-background-size:120px 14px;-moz-background-size:120px 14px;background-size:120px 14px}.hdpi.pac-logo:after{background-image:url(https://maps.gstatic.com/mapfiles/api-3/images/powered-by-google-o

In [35]:
def guru_scraper(page, job_title, type):

    global guru_url
    guru_url = f"https://www.guru.com/d/{type}/skill/{job_title}/pg/{page}/"
    html = scrape_page(guru_url)

    if len(html.find_all("div", class_= "record jobRecord")) == 0:
        raise  KeyError

    return html


def fivver_scraper(page, job_title):

    global fivver_url
    fivver_url = f"https://www.fiverr.com/categories/data/{job_title}?source=pagination&page={page}"
    html = scrape_page(fivver_url)

    if "Well, this isn't what you were looking for" in html.find("h3", class_= "title"):
        raise  KeyError

    return html


def pph_scraper(page, job_title, type):

    global peopleperhour_url

    if type == "hire-freelancers":
        peopleperhour_url = f"https://www.peopleperhour.com/hire-freelancers/{job_title.replace('-', '+')}?page={page}&ref=search"
        html, new_url = scrape_page(peopleperhour_url, retrieve_new_url= True)
        loaded_page = int(str(new_url.split("&")[0]).rsplit("=", 1)[-1])

    else:
        peopleperhour_url = f"https://www.peopleperhour.com/{type}/{job_title.replace('-', '+')}?page={page}"
        html, new_url = scrape_page(peopleperhour_url, retrieve_new_url= True)
        loaded_page = int(new_url.rsplit("=", 1)[-1])

    if page != loaded_page:
        raise  KeyError

    return  html

# For sorry we won't analyze freelancer.com freelancers profiles data becuase
# freelancer.com search is awful so we will only search the jobs data.

def freelancer_scraper(page, job_title):

    global freelancer_url
    freelancer_url = f"https://www.freelancer.com/jobs?search_keyword={job_title.replace('-', '+')}&page={page}"
    
    return scrape_page(freelancer_url)

Now we will try to calculate how much did each scraper take to collect the data

In [5]:
get_variable_name = lambda var: [name for name in globals() if globals()[name] is var][0]

def get_runtime(func, *args, **kwargs):

    t1 = time.perf_counter()
    output = func(*args, **kwargs)
    t2 = time.perf_counter()

    run_time = t2 - t1

    print("{} took about {:.2f}s to run.\n".format(
        get_variable_name(func).replace("_", " ").title(), run_time))

    return  output

Now we will just try to scrape **only one page** from each scraper *(temporary action)* to just test the data extractors.

## <center><strong>Collecting the Websites <span style = "color: #4895e0"> HTML</span>

In [6]:
soups = {"soup_name": [],
         "soup_job_type": [],
         "soup_job_title": [],
         "soup_page":  [],
         "soup": []}

In [14]:
pph_html = pph_scraper(1_00000, "data-entry", "freelance-jobs")

In [8]:
# If the job type is services in upwork the pages paremeter will refer to how many pages 
# will render at once while the other scrapers will render the spesfic spesfied page only.

def get_all_pages_soups(pph_type, upwork_type, guru_type, max_page: int= 999) -> dict:

    print("Strating ...")
    flag = False
    
    for i in count(0):

        freelancer_type = "jobs"
        fivver_type = "services"
        
        page = i + 1

        if flag:
            break

        print(f"Loading page {page} of each website ...")


        for data_job_title in data_jobs_titles:

            lines = [f'freelancer_html = freelancer_scraper({page}, "{data_job_title}")',
                     f'pph_html = pph_scraper({page}, "{data_job_title}", "{pph_type}")']
            
            # f'fivver_html = fivver_scraper({page}, "{data_job_title}")',
            # f'upwork_html = fivver_scraper({page}, "{data_job_title}")',
            # f'guru_html = fivver_scraper({page}, "{data_job_title}")',

            if len(lines) < 1 or page > max_page:

                print("Finished ...")
                flag = True

            if flag:
                break
            

            for line_idx, line in enumerate(lines):

                equall_sign_idx = line.find('=') - 1
                first_underscore_idx = line.find('_')

                try:
                    exec(line)
                    print(line)
                    
                    soups["soup_job_title"].append(data_job_title)
                    soups["soup_name"].append(line[:equall_sign_idx])
                    soups["soup_page"].append(page)
                    soups["soup"].append(eval(str(line[:equall_sign_idx])))
                    soups["soup_job_type"].append(eval(f'{line[:first_underscore_idx]}_type'))


                except Exception as e:

                    # print(e)
                    del lines[line_idx]
                    # if len(lines) < 1 or page >max_page:

                    #     print(e)
                    #     print("Finished ...")
                    #     break

                    continue

In [9]:
get_all_pages_soups("freelance-jobs", "jobs", "jobs", max_page= 2)

Strating ...
Loading page 1 of each website ...
freelancer_html = freelancer_scraper(1, "data-entry")
pph_html = pph_scraper(1, "data-entry", "freelance-jobs")
freelancer_html = freelancer_scraper(1, "data-processing")
pph_html = pph_scraper(1, "data-processing", "freelance-jobs")
freelancer_html = freelancer_scraper(1, "data-engineering")
pph_html = pph_scraper(1, "data-engineering", "freelance-jobs")
Loading page 2 of each website ...
freelancer_html = freelancer_scraper(2, "data-entry")
pph_html = pph_scraper(2, "data-entry", "freelance-jobs")


KeyboardInterrupt: 

In [9]:
max_length = max(len(lst) for lst in soups.values())
soups = {key: lst + [float('nan')] * (max_length - len(lst)) for key, lst in soups.items()}

df = pd.DataFrame(soups)

{'col1': [1, 2, 3, 5, 5], 'col2': [1, 2, 3, 9, nan]}

In [8]:
print("Printing the title of each page to check if it's loaded correctly.\n")

for soup in soups["soup"]:
    print("-\t", soup.title.text.replace("\n", "").replace("\t", ""))

soups = pd.DataFrame(soups)

Printing the title of each page to check if it's loaded correctly.

-	 Freelance Data-analytics Jobs - Upwork
-	 Data Analytics Services Online | Fiverr
-	 Machine Learning Freelance Jobs Online - Guru
-	 Get Data Analytics Offers & Services | PeoplePerHour
-	 Users Jobs, Employment | Freelancer


Looks like everything is working as expected now let's start reel work by extracting the HTML tags data from the scraped<br>
pages.<br> <br>

But we will need to collect the data from each one of those pages with unique way further more we need to optimize the<br>
data extractor to collect the data from other pages from the same site like "jobs" site and "profiles" sites on upwork.<br>

In [67]:
def get_cards_links(soup_name, tag, link_template,
                    link_attr, tag_class, attr= "class") -> list:

    soup = soups[soups["soup_name"] == soup_name]["soup"].values[0]

    cards = soup.find_all(tag, attrs= {attr: tag_class})
    cards_ids = []

    if link_attr != "":

        for card in cards:

            card_id = link_template.replace("###", card[link_attr])

            if len(card_id) > len(link_template):
                cards_ids.append(card_id)

        return  cards_ids

    else:
        return  cards

In [85]:
# We can't collect guru data this wat becuase it has slightly deffrint case in terms
# of collecting the jobs URLs

## --------------------------Upwork--------------------------
upwork_gigs_links = get_cards_links('upwork_soup', 'a', 'https://upwork.com###', 'href',
                                        'up-n-link project-tile up-card m-0 p-0 up-card-section up-card-hover project-tile_flexible')

upwork_profiles_links = get_cards_links('upwork_soup', 'div', 'https://www.upwork.com/search/profiles/?profile=###', 'data-test-key',
                                         'up-card-section up-card-hover')

upwork_jobs_parents = get_cards_links('upwork_soup', 'h3', 'https://www.upwork.com###', '', 'my-0 p-sm-right job-tile-title')
upwork_jobs_links = []

for parent in upwork_jobs_parents:
    upwork_jobs_links.append("https://www.upwork.com" + parent.find('a', class_= "up-n-link")['href'])
## ----------------------------------------------------------


## --------------------------Fivver--------------------------
fivver_gigs_links = get_cards_links('fivver_soup', 'a', 'https://www.fiverr.com###', 'href', 'tbody-5 p-t-4 YLycza2 u9KHmsf')
## ----------------------------------------------------------


## ---------------------People per hour----------------------
pph_services_links = get_cards_links("peopleperhour_soup", "a", "https://www.peopleperhour.com###", "href", 
                                     "card__title-wrapper⤍FreelancerCard⤚2-NWk")

pph_freelancers_links = get_cards_links("peopleperhour_soup", "a", "https://www.peopleperhour.com###", "href",
                                        "card__title-wrapper⤍FreelancerCard⤚2-NWk")

pph_jobs_links = get_cards_links("peopleperhour_soup", "a", "https://www.peopleperhour.com###", "href",
                                 "item__url⤍ListItem⤚20ULx")
## ----------------------------------------------------------


## ------------------------Guru------------------------------
guru_cards_parents_profiles = get_cards_links("guru_soup", "h2", "https://guru.com###", "",
                                              "jobRecord__title jobRecord__title--changeVisited")

guru_cards_parents_jobs = get_cards_links("guru_soup", "h2", "https://guru.com###", "",
                                          "jobRecord__title jobRecord__title--changeVisited")
## ----------------------------------------------------------


## ----------------------Freelancer---------------------------
freelancer_jobs_links = get_cards_links("freelancer_soup", "a", "https://www.freelancer.com###", "href",
                                        "LinkElement ng-star-inserted")

freelancer_freelancers_links = get_cards_links("freelancer_soup", "a", "https://www.freelancer.com###", "href",
                                              "LinkElement ng-star-inserted")

In [ ]:
def get_upwork_links(job_type: str) -> list:  

    card_tag = "a"
    link_template = "https://www.upwork.com###"
    link_attr = "href"

    if job_type == "profiles":

        card_tag = "div"
        card_tag_class = "up-card-section up-card-hover"
        link_template = "https://www.upwork.com/search/profiles/?q=profiles&profile=###"
        link_attr = "data-test-key"
    
    elif job_type == "jobs":
        card_tag_class = "up-n-link"


    elif job_type == "services":
        card_tag_class = "up-n-link project-tile up-card m-0 p-0 up-card-section up-card-hover project-tile_flexible"

    else:
        raise Exception("Please input correct work type from ['profiles', 'jobs', 'services'].")


    soup = soups[soups["soup_name"] == "upwork_soup"]["soup"][0]

    cards = soup.find_all(card_tag, class_= card_tag_class)
    cards_ids = []

    for card in cards:
        
        card_id = link_template.replace("###", card[link_attr])
        
        if len(card_id) > len(link_template):
            cards_ids.append(card_id)

    return  cards_ids


def get_fivver_links() -> list:

    card_tag = "div"
    card_tag_class = "basic-gig-card"
    link_template = "https://www.fiverr.com###"
    link_attr = "href"

    cards = soup.find_all(card_tag, class_= card_tag_class)
    cards_ids = []
    
    for card in cards:
        
        card_id = link_template.replace("###", card[link_attr])
        
        if len(card_id) > len(link_template):
            cards_ids.append(card_id)

    return  cards_ids

In [ ]:
def get_guru_links(job_type: str) -> list:  

    soup = soups[soups["soup_name"] == "guru_soup"]["soup"][2]
    link_template = "https://www.guru.com###"

    if job_type == "freelancers":
        cards_parents = soup.find_all(
            'h2', class_= 'serviceListing__title serviceListing__title--dark')

        cards = []

        for parent in cards_parents:
            card = parent.find('a')
            cards.append(card)

    elif job_type == "jobs":
        cards_parents = soup.find_all(
            'h2', class_ = 'jobRecord__title jobRecord__title--changeVisited')

        cards = []

        for parent in cards_parents:
            card = parent.find('a')
            cards.append(card)

    cards_ids = []

    for card in cards:
        card_id = link_template.replace("###", card['href'])
        cards_ids.append(card_id)

    return  cards_ids

In [ ]:
get_guru_links('jobs')